## Example for implementation of Spectrum estimation based on the Order Tracking

My objective is to increase the coverage for machines with unstable operational conditions (load and torque). I have bumped into a crane with an unstable operational speed. For this case, I want to use Order Tracking, allowing further utilization of the already implemented spectrum analysis tools. Later, I think that we can use this technique for other equipment too. 

In [ ]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib notebook
from scipy.signal import ellip, firwin, filtfilt, sosfreqz, sosfiltfilt
from scipy.ndimage.filters import uniform_filter1d
from scipy.fftpack import fft, fftfreq, next_fast_len
from scipy.interpolate import interp1d

The magnetic file names

In [ ]:
files = glob('Buffer Crane - 6022c7d2dbf9880001a364e9/*magnetic*')
files[:5]

In [ ]:
def plotFiltResponse(sos, fs):
    w, h = sosfreqz(sos, fs=fs, worN=200000)
    dispCond = w < 20
    plt.figure(figsize=[15, 5])
    plt.plot(w[dispCond], abs(h[dispCond]))
    plt.title('Filter frequency response')
    plt.xlabel('Frequency [radians / second]')
    plt.ylabel('Amplitude')
    plt.grid()
    plt.show()

Magnetic file where the engine is on

In [ ]:
filesMagnetic = []
for file in files:
    x = np.load(file)
    if 'sos' not in locals():
        fs = 1 / x[0, 1]
        sos = ellip(6, 1, 40, 2, btype='high', output='sos', fs=fs)
#         plotFiltResponse(sos, fs)
    temp = sosfiltfilt(sos, x[1, :])  
    # looking for a file with activity
    cond1 = (np.percentile(temp, 80) - np.percentile(temp, 20)) > 0.5
    cond2 = x[1, :].mean() < 100
    if (cond1 & cond2):
#         plt.figure(figsize=[15, 5])
#         plt.plot(x[0, :], x[1, :])
#         plt.title(file.split('/')[-1])
#         plt.show()
        filesMagnetic.append(file)
len(filesMagnetic)

In [ ]:
def orderTrack(xMagnetic, x, t, fs, b=firwin(9, 0.1), isPlot=False):
    #this function implements the order-tracking and the speed estimation (which actually should be in another function)
    crossLocs = np.where(np.diff(np.sign(xMagnetic - np.median(xMagnetic))))[0]
    # avoiding fault detection of crossing due to noise
    crossLocs = crossLocs[np.where(np.diff(crossLocs)>30)]
    #avoiding sampling of half-cycle, which leads to leakage
    if crossLocs.shape[0] % 2:
        crossLocs = crossLocs[:-1]

    # locations of the estimated speed
    speedLocs = (crossLocs[:-1] + crossLocs[1:]) / 2
    speedTime = speedLocs / fs
    # could skip estimation of the speed but it is nise to have it
    speed = fs / np.diff(crossLocs) / 2
    # due to higher harmonics, we have to filter. 
#     Here, I don't care that the distance between the samples is not constant
    filtSpeed = filtfilt(b, 1, speed, padtype='even')
    
    # trimming the samples with a non-defined speed 
    cond = (t > speedTime[0]) & (t <speedTime[-1])
    x = np.atleast_2d(x)[cond, :]
    t = t[cond]
    
    #interpolation of speed
    interpSpeedFun = interp1d(speedTime, filtSpeed, axis=0)
    interpSpeed = interpSpeedFun(t)
    
    #phase in cycle units
    phase = np.cumsum(interpSpeed) / fs 
    phase -= phase[0]
    # avoiding aliasing by sampling at least as fast as previously
    minDphase = np.min(speed) / fs
    # finding the increment for a fast fft calculation
    dPhase = phase[-1] / next_fast_len(np.ceil(phase[-1] / minDphase).astype(int))
    resampledPhase = np.arange(0, phase[-1], dPhase)
    interpFun = interp1d(phase, x, axis=0)
    resampledX = interpFun(resampledPhase)
    
    if isPlot:
    
        plt.figure(figsize=[15, 5])
        plt.plot(speedTime, speed)
        plt.plot(speedTime, filtSpeed)
        plt.legend(['Gross speed estimation', 'Filtered speed estimtaton'])
        plt.xlabel('Time [sec]')
        plt.ylabel('Shaft speed [Cycles / second]')
        plt.plot()
        plt.show()
        
        plt.figure(figsize=[15, 5])
        plt.plot(t, xMagnetic)
        plt.title(file.split('/')[-1])
        plt.show()

    return resampledPhase, resampledX, t, phase, interpSpeed, x

Packing the data of the vibration signals with magnetic signal

In [ ]:
for fileMagnetic in filesMagnetic:
    print(fileMagnetic)
    filesVib = glob(fileMagnetic.split('_plane_')[0] + '*vibration*')
    xVibration = np.concatenate([np.atleast_2d(np.load(file)[1, :]) for file in filesVib]).T
    
    x = np.load(fileMagnetic)
    t = x[0, :]
    fs = 1 / (t[1] - t[0])
    xMagnetic = x[1, :]
    
    files = [fileMagnetic] + filesVib
    x = np.concatenate([np.atleast_2d(xMagnetic).T, xVibration], axis=1)
    x -= x.mean(axis=0)
    
    resampledPhase, resampledX, t, phase, speed, xTrimmed = orderTrack(xMagnetic, x, t, fs)

    plt.figure(figsize=[15, 5])
    plt.plot(t, speed)
    plt.ylabel('Shaft speed [cycle / sec]')
    plt.xlabel('Time [sec]')
    plt.grid()
    plt.show()
    
    for iSig, file in enumerate(files):
        plt.figure(figsize=[15, 5])

        f = fftfreq(resampledX.shape[0], d=(resampledPhase[1] - resampledPhase[0]))
        xWind = resampledX[:, iSig] * np.hanning(resampledX.shape[0])
        plt.plot(f[f > 0], np.log(np.abs(fft(xWind, n=resampledX.shape[0])))[f > 0])

        f = fftfreq(100000, d=1 / fs)
        xWind = x[:, iSig] * np.hanning(x.shape[0])
        plt.plot(f[f > 0] / speed.mean(), np.log(np.abs(fft(xWind, n=100000)))[f > 0])

        plt.title(file.split('/')[-1])
        plt.xlabel('Shaft Speed Order [-] / Frequency normalized by mean speed')
        plt.ylabel('DFT-Absolute Value - log scale')
        plt.xlim([0, 10])
        plt.xticks(np.arange(10))
        plt.legend(['Order tracking', 'Original DFT'])
        plt.grid()
        plt.show()

For non-stationary cases, we can see an improvement in the spectrum elements' decomposition. Besides that, we can see the following improvements:
1. Reduction in the dc noise (probably from the hardware). Why?
2. Reduction in the leakage (this one is yet pending improvement). Have been seen in multiple locations (see the last magnetic signal spectrum). This probably happens due to the trimming of the signal at the precise multiplication of the cycle.
3. Concentration of the energy in the integration points, located at the harmonics of the shaft speed order.

Points to improve:
1. Trimming of the signal where the machine is off.
2. figure out why we have DC.